In [32]:
! pip install category_encoders



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
# # Import necessary libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.linear_model import LogisticRegression, RidgeClassifier
# from sklearn.ensemble import StackingClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from category_encoders.woe import WOEEncoder  # For WoE encoding
# from sklearn.feature_selection import mutual_info_classif
# import phik
# from sklearn.metrics import roc_auc_score, accuracy_score
# import numpy as np

# # Load your train and test datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Separate target variable from features
# y = train_df['Exited']
# X = train_df.drop(columns=['Exited', 'id'])  # Drop 'id' and target 'Exited' from the training features
# X_test = test_df.drop(columns=['id'])  # Drop 'id' from the test features

# # Step 1: Divide the train data into two parts (train/validation split)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# # Step 2: Feature Engineering on Each Column Separately

# # List of columns to be processed
# categorical_columns = ['Geography', 'Gender']  # Example categorical columns
# numerical_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

# # WoE Encoding for categorical columns
# woe_encoder = WOEEncoder(cols=categorical_columns)

# # SimpleImputer for missing values and StandardScaler for numerical columns
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
#     ('scaler', StandardScaler())  # Standard scaling for numerical columns
# ])

# # Apply WoE encoding to categorical columns
# categorical_transformer = Pipeline(steps=[
#     ('woe', woe_encoder)
# ])

# # Combine preprocessing steps using ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_columns),
#         ('cat', categorical_transformer, categorical_columns)
#     ]
# )

# # Step 3: Model Selection and Training

# # Define base models for stacking
# log_reg = LogisticRegression(solver='liblinear', max_iter=1000)
# ridge_clf = RidgeClassifier()

# # Combine models using stacking
# base_models = [
#     ('ridge', ridge_clf),
#     ('log_reg', log_reg)
# ]

# stack_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(solver='liblinear', max_iter=1000))

# # Create a pipeline with preprocessing and the model
# pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('stack_model', stack_model)])

# # Step 4: Train the Model and Evaluate on the Validation Set
# pipeline.fit(X_train, y_train)

# # Predict on validation set
# y_val_pred = pipeline.predict(X_val)
# val_roc_auc = roc_auc_score(y_val, y_val_pred)
# val_accuracy = accuracy_score(y_val, y_val_pred)

# print(f'Validation ROC-AUC: {val_roc_auc}')
# print(f'Validation Accuracy: {val_accuracy}')

# # Step 5: Hyperparameter Tuning for Improvement
# param_grid = {
#     'stack_model__final_estimator__C': [0.1, 1, 10, 100],
#     'stack_model__final_estimator__penalty': ['l1', 'l2']
# }

# # Use GridSearchCV to find optimal hyperparameters
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')
# grid_search.fit(X_train, y_train)

# # Best model from grid search
# best_model = grid_search.best_estimator_
# print(f"Best parameters: {grid_search.best_params_}")

# # Predict with the best model
# y_val_best_pred = best_model.predict(X_val)
# best_val_roc_auc = roc_auc_score(y_val, y_val_best_pred)
# best_val_accuracy = accuracy_score(y_val, y_val_best_pred)

# print(f'Best Validation ROC-AUC: {best_val_roc_auc}')
# print(f'Best Validation Accuracy: {best_val_accuracy}')

# # Step 6: Predictions on Test Data with Best Model
# y_test_pred = best_model.predict_proba(X_test)[:, 1]  # Probability for class "Exited"

# # Prepare the submission file
# submission = test_df[['id']].copy()
# submission['Exited'] = y_test_pred
# submission.to_csv('Final.csv', index=False)

# print("Final file created successfully.")

# # Step 7: Feature Importance - Mutual Information (Optional for feature selection)
# X_train_transformed = preprocessor.fit_transform(X_train, y_train)  # Apply transformation for feature selection
# mi_scores = mutual_info_classif(X_train_transformed, y_train)
# mi_scores_series = pd.Series(mi_scores, index=numerical_columns + categorical_columns)
# print(f"Mutual Information Scores:\n{mi_scores_series.sort_values(ascending=False)}")

# # Step 8: PhiK Correlation (Optional for non-linear relationships)
# phik_matrix = pd.DataFrame(X_train).phik_matrix(interval_cols=numerical_columns)
# print(f"PhiK Correlation Matrix:\n{phik_matrix}")


In [36]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, accuracy_score
from category_encoders.woe import WOEEncoder

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Separate target variable from features
y = train_df['Exited']
X = train_df.drop(columns=['Exited', 'id'])  # Drop 'id' and target 'Exited' from the training features
X_test = test_df.drop(columns=['id'])  # Drop 'id' from the test features

# List of categorical and numerical columns
categorical_columns = ['Geography', 'Gender']  # Categorical columns
numerical_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

# Step 1: Apply WoE encoding to categorical features
woe_encoder = WOEEncoder(cols=categorical_columns)
X_train_woe = woe_encoder.fit_transform(X, y)  # WoE encode the train data
X_test_woe = woe_encoder.transform(X_test)  # WoE encode the test data

# Step 2: Split the encoded data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_woe, y, test_size=0.3, random_state=42, stratify=y)

# Step 3: Feature Engineering on Each Column Separately

# Create a copy of the dataset to perform feature engineering
X_train_fe = X_train.copy()
X_val_fe = X_val.copy()
X_test_fe = X_test_woe.copy()

# 3.1 Polynomial Features (Second-degree interactions)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_features_train = poly.fit_transform(X_train_fe[numerical_columns])
poly_features_val = poly.transform(X_val_fe[numerical_columns])
poly_features_test = poly.transform(X_test_fe[numerical_columns])

# Add polynomial features to the original dataset
poly_feature_names = poly.get_feature_names_out(numerical_columns)
X_train_fe = pd.concat([X_train_fe, pd.DataFrame(poly_features_train, columns=poly_feature_names)], axis=1)
X_val_fe = pd.concat([X_val_fe, pd.DataFrame(poly_features_val, columns=poly_feature_names)], axis=1)
X_test_fe = pd.concat([X_test_fe, pd.DataFrame(poly_features_test, columns=poly_feature_names)], axis=1)

# 3.2 Group Statistics (Example: Mean balance per Geography group)
geography_balance_mean = X_train_fe.groupby('Geography')['Balance'].mean()
X_train_fe['Geography_Balance_Mean'] = X_train_fe['Geography'].map(geography_balance_mean)
X_val_fe['Geography_Balance_Mean'] = X_val_fe['Geography'].map(geography_balance_mean)
X_test_fe['Geography_Balance_Mean'] = X_test_fe['Geography'].map(geography_balance_mean)

# Additional feature engineering on Balance and EstimatedSalary (e.g., creating groups)
balance_bins = [-np.inf, 0, 100000, 200000, np.inf]
salary_bins = [-np.inf, 50000, 100000, 150000, np.inf]
X_train_fe['Balance_Group'] = pd.cut(X_train_fe['Balance'], bins=balance_bins, labels=['Low', 'Medium', 'High', 'Very High'])
X_val_fe['Balance_Group'] = pd.cut(X_val_fe['Balance'], bins=balance_bins, labels=['Low', 'Medium', 'High', 'Very High'])
X_test_fe['Balance_Group'] = pd.cut(X_test_fe['Balance'], bins=balance_bins, labels=['Low', 'Medium', 'High', 'Very High'])

X_train_fe['Salary_Group'] = pd.cut(X_train_fe['EstimatedSalary'], bins=salary_bins, labels=['Low', 'Medium', 'High', 'Very High'])
X_val_fe['Salary_Group'] = pd.cut(X_val_fe['EstimatedSalary'], bins=salary_bins, labels=['Low', 'Medium', 'High', 'Very High'])
X_test_fe['Salary_Group'] = pd.cut(X_test_fe['EstimatedSalary'], bins=salary_bins, labels=['Low', 'Medium', 'High', 'Very High'])

# Step 4: Preprocessing and Model Training

# Update the list of numerical columns with the new features
numerical_columns += poly_feature_names.tolist() + ['Geography_Balance_Mean']

# WoE Encoding for categorical columns
woe_encoder = WOEEncoder(cols=categorical_columns)

# SimpleImputer for missing values and StandardScaler for numerical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    ('scaler', StandardScaler())  # Standard scaling for numerical columns
])

# Apply WoE encoding to categorical columns
categorical_transformer = Pipeline(steps=[
    ('woe', woe_encoder)
])

# Combine preprocessing steps using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Define base models for stacking
log_reg = LogisticRegression(solver='liblinear', max_iter=1000)
ridge_clf = RidgeClassifier()
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
gb_clf = GradientBoostingClassifier(n_estimators=200, random_state=42)

# Combine models using stacking
base_models = [
    ('ridge', ridge_clf),
    ('log_reg', log_reg),
    ('rf', rf_clf),
    ('gb', gb_clf)
]

stack_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(solver='liblinear', max_iter=1000))

# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('stack_model', stack_model)])

# Train the model
pipeline.fit(X_train_fe, y_train)

# Step 5: Evaluate on validation set and make predictions on test set
y_val_pred = pipeline.predict(X_val_fe)
val_roc_auc = roc_auc_score(y_val, y_val_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Output validation scores
print(f'Validation ROC-AUC: {val_roc_auc}')
print(f'Validation Accuracy: {val_accuracy}')

# Step 6: Predictions on test data with custom rounding logic for Exited values
y_test_pred = pipeline.predict_proba(X_test_fe)[:, 1]  # Probability for class "Exited"

# Apply custom rounding logic
y_test_pred_custom = []
for pred in y_test_pred:
    if pred >= 0.75:
        y_test_pred_custom.append(1)
    elif 0.5 <= pred < 0.75:
        y_test_pred_custom.append(np.round(pred, 2))  # Round to 2 decimal places if between 0.5 and 0.74
    else:
        y_test_pred_custom.append(pred)  # Keep the rest unchanged

# Convert the list back to a NumPy array
y_test_pred_custom = np.array(y_test_pred_custom)

# Step 7: Prepare the submission file
submission = test_df[['id']].copy()
submission['Exited'] = y_test_pred_custom  # Add the custom-rounded predicted probabilities
submission.to_csv('SubmissionFinal.csv', index=False)

# Notify that the file was successfully created
print("SubmissionFinal.csv file created successfully.")


ValueError: Data must be 1-dimensional, got ndarray of shape (3, 2) instead